In [45]:
#importinng lib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras

In [6]:
#importing dataset
data =pd.read_csv('train.csv') #dataset file is saved as 'train.csv'
data.head() #printing first few rows of data

,id,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.0,0.0,0.004883,0.000000,0.025391
1,2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.0,0.0,0.000977,0.039062,0.022461
2,3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.0,0.0,0.000000,0.020508,0.002930
3,5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.0,0.0,0.017578,0.000000,0.047852
4,6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.031250


In [7]:
#moving 'species' columns to target(dependent variable) 
target = data.pop('species')

#dropping 'id' column as it is not required
data = data.drop(labels = ['id'], axis = 1)

In [8]:
#encoding the target by using LabelEncoder from sklearn

from sklearn.preprocessing import LabelEncoder #importing liabrary

enc = LabelEncoder().fit(target) #fitting target into LabelEncoder
target = enc.transform(target)  

Scaling is important while training the dataset<br>
Here, I am using StandardScaler from sklearn <br>

In [13]:

from sklearn.preprocessing import StandardScaler #importing liabrary

sc_x = StandardScaler().fit(data)
X = sc_x.transform(data)

In [16]:
print(X.shape) #printing shape of independent variables
print(target.shape) #printing shape of dependent variable

(990, 192)
(990,)


In [44]:
#spliting dataset into trainnig & testing set

from sklearn.model_selection import train_test_split #impoting liabraries

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.2) 
#here testing set size will be 20% of training set

Here we are using keras Sequential model <br>
In sequential model data flows in forward direction <br>


In [17]:
# importing model from keras
model = keras.models.Sequential()

In [20]:
#adding first hidden layer with 300 nodes, & using activation function as RECTIFIED LINEAR function
#Dimension of input layer is 192(no of features), as we have 192 feature in our dataset
model.add(keras.layers.Dense(300, input_dim = 192, activation = tf.nn.relu))

In [21]:
#adding second hidden layer to our model, with 300 nodes & RECTIFIED LINEAR activation funcation
model.add(keras.layers.Dense(300, activation = tf.nn.relu))

In [22]:
#adding output layer to the model, with 99(no of categories) nodes & SOFTMAX as activation function
model.add(keras.layers.Dense(99, activation = tf.nn.softmax))

In [23]:
#compiling the model with 'ADAM' as optimizer, 'SPARSE_CATEGORICAL_CROSSENTROPY'  as loss function & using accuracy matrix
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy']
            
)

In [36]:
#training the model, with 3 iterations 
model.fit(X_train , y_train, epochs = 5)

Epoch 1/5
792/792 [==============================] - 0s 183us/step - loss: 0.0795 - acc: 0.9899
Epoch 2/5
792/792 [==============================] - 0s 155us/step - loss: 0.0412 - acc: 0.9987
Epoch 3/5
792/792 [==============================] - 0s 185us/step - loss: 0.0259 - acc: 0.9987
Epoch 4/5
792/792 [==============================] - 0s 192us/step - loss: 0.0238 - acc: 0.9962
Epoch 5/5
792/792 [==============================] - 0s 188us/step - loss: 0.0201 - acc: 0.9962


In [38]:
#evaluating model

val_loss, val_acc = model.evaluate(X_test, y_test)

print('losses: %f', val_loss)
print('accuracy: %f', val_acc)

198/198 [==============================] - 0s 78us/step
losses: %f 0.024660144490425032
accuracy: %f 1.0


In [39]:
#importing test dataset
data_test = pd.read_csv('test.csv') #test dataset is stored as test.csv
test_id = data_test.pop('id')     #popping 'id' column to test_id

In [40]:
data_test = StandardScaler().fit(data_test).transform(data_test) #scaling the dataset

In [41]:
predictions = model.predict_proba(data_test)  #predicting the species

Submission file format is like array of probbility of occurance of target, <br>
Using predict_proba, it'll give array of (no of examples * no of features) <br>
where each value in a row states probability of sample to be in a particular column <br>

In [42]:
#converting the list in dataframe form, using our prediction array as data to dataframe, 
#idex of dataframe as test_id's & columns as features
result = pd.DataFrame(predictions, index = test_id, columns = enc.classes_)

In [43]:
#converting dataframe into csv file, 
result.to_csv('result.csv')